In [1]:
import sys, pickle, json, os, re
import numpy as np
from collections import defaultdict
from pprint import pprint
from typing import List, Dict, Any
sys.path.append('../src/')
sys.path.append("../../scripts/")
from utils import find_repo_root

import matplotlib.pyplot as plt
from plt_config import use_latex, TU_GREY, TU_RED
use_latex()
project_root_dir = find_repo_root()
print("Project root directory:", project_root_dir)

Project root directory: /home/etschgi1/REPOS/Masterarbeit


In [23]:
ray_folder = "/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_testing/tune_logs/MGNN_hyp_small_full_mat_loss.py"
trials = []
for folder in os.listdir(ray_folder):
    sub_folder_path = os.path.join(ray_folder, folder)
    if os.path.exists(os.path.join(sub_folder_path, "eval_res.json")):
        with open(os.path.join(sub_folder_path, "eval_res.json")) as f:
            eval_res = json.load(f)
        with open(os.path.join(sub_folder_path, "params.json")) as f:
            params = json.load(f)
        trials.append((params, eval_res))

In [24]:
trials.sort(key=lambda x:x[1]["cummulative_stats"]["iterations"]["mean"])
for trial in trials: 
    print(trial[0])
    print(trial[1]["cummulative_stats"])
    print("---")

{'batch_size': 8, 'data_aug_factor': 1.8283773645141828, 'edge_threshold_val': 2.733101699166326, 'grace_epochs': 5, 'hidden_dim': 512, 'loss_on_full_matrix': True, 'lr': 0.000633634973297049, 'lr_cooldown': 2, 'lr_factor': 0.5, 'lr_min': 1e-06, 'lr_patience': 3, 'lr_threshold': 0.001, 'message_net_dropout': 0.103867632797979, 'message_net_layers': 3, 'message_passing_steps': 2, 'num_epochs': 30, 'weight_decay': 0.0004561806598773125}
{'iterations': {'mean': 11.48, 'std': 0.9431860898041278}, 'energy_abs': {'mean': -1254.8999718631876, 'std': 37.812301352121985}, 'energy_rel': {'mean': -0.7183590438076577, 'std': 0.011168065734956862}, 'diis': {'mean': 0.20912042996427752, 'std': 0.017106338553774407}, 'rmse': {'mean': 0.0076058432116162256, 'std': 0.0006337539396056591}}
---
{'batch_size': 8, 'data_aug_factor': 2.4588815931150743, 'edge_threshold_val': 3.870510473622333, 'grace_epochs': 5, 'hidden_dim': 256, 'loss_on_full_matrix': True, 'lr': 0.00023052638210855868, 'lr_cooldown': 2, 

In [32]:
table_data = []
for i, (params, metrics) in enumerate(trials):
    metrics = metrics["cummulative_stats"]
    row = {
        'ID': i,
        'Iterations': f"{float(metrics['iterations']['mean']):.1f}({float(metrics['iterations']['std']):.1f})",
        r"$\Delta E_\text{HF}$": f"{float(metrics['energy_abs']['mean']):.1f}({float(metrics['energy_abs']['std']):.1f})",
        r"$\delta E_\text{HF}$": f"{float(metrics['energy_rel']['mean']):.3f}({float(metrics['energy_rel']['std']):.3f})",
        'DIIS error': f"{float(metrics['diis']['mean']):.3f}({float(metrics['diis']['std']):.2f})",
        r"$\overline{\text{RMSE}}$": f"{float(metrics['rmse']['mean']):.4f}({float(metrics['rmse']['std']):.4f})"
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data)

latex = df.to_latex(index=False)
print(latex)

\begin{tabular}{rlllll}
\toprule
ID & Iterations & $\Delta E_\text{HF}$ & $\delta E_\text{HF}$ & DIIS error & $\overline{\text{RMSE}}$ \\
\midrule
0 & 11.5(0.9) & -1254.9(37.8) & -0.718(0.011) & 0.209(0.02) & 0.0076(0.0006) \\
1 & 11.6(0.7) & -1246.3(40.2) & -0.713(0.012) & 0.206(0.02) & 0.0078(0.0006) \\
2 & 11.7(0.8) & -1188.4(36.2) & -0.680(0.010) & 0.190(0.02) & 0.0079(0.0006) \\
3 & 13.1(1.1) & -1313.8(46.2) & -0.752(0.014) & 0.230(0.02) & 0.0084(0.0005) \\
4 & 13.2(1.2) & -1314.4(41.9) & -0.752(0.012) & 0.234(0.02) & 0.0088(0.0005) \\
5 & 13.3(1.1) & -1298.0(42.3) & -0.743(0.012) & 0.232(0.02) & 0.0090(0.0005) \\
6 & 13.6(1.3) & -1315.1(44.4) & -0.753(0.014) & 0.235(0.02) & 0.0089(0.0005) \\
7 & 13.9(1.1) & -1324.5(46.8) & -0.758(0.014) & 0.238(0.02) & 0.0088(0.0006) \\
8 & 14.5(1.4) & -1345.4(41.3) & -0.770(0.012) & 0.244(0.02) & 0.0090(0.0005) \\
9 & 14.7(1.2) & -1349.0(50.6) & -0.772(0.016) & 0.248(0.02) & 0.0091(0.0005) \\
\bottomrule
\end{tabular}

